# 1. Music Industry Project


## load&connectwithdatabase

In [1]:
%%capture
%load_ext sql
%sql sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db

## Overview of the Data

In [2]:
%sql SELECT name FROM sqlite_master WHERE type='table';

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


## Revenue by Country

In [3]:
%%sql
SELECT BillingCountry, sum(Total) as income from invoices
GROUP BY BillingCountry
ORDER BY income DESC;

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

BillingCountry,income
USA,523.06
Canada,303.96
France,195.1
Brazil,190.1
Germany,156.48
United Kingdom,112.86
Czech Republic,90.24
Portugal,77.24
India,75.26
Chile,46.62


This analysis shows that North America is the continent that generates the most revenue, suggesting strong purchasing power and higher consumption intensity.

## Top genres in USA

In [4]:
%%sql SELECT BillingCountry, g.Name, ROUND(SUM(t.UnitPrice*Quantity)) AS Sum
    FROM invoices
JOIN main.invoice_items ii on invoices.InvoiceId = ii.InvoiceId
JOIN main.tracks t on ii.TrackId = t.TrackId
JOIN main.genres g on t.GenreId = g.GenreId
WHERE BillingCountry = 'USA'
GROUP BY BillingCountry, g.Name
ORDER BY Sum DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

BillingCountry,Name,Sum
USA,Rock,155.0
USA,Latin,90.0
USA,Metal,63.0
USA,Alternative & Punk,50.0
USA,TV Shows,28.0
USA,Jazz,22.0
USA,Comedy,16.0
USA,Blues,15.0
USA,Drama,12.0
USA,R&B/Soul,12.0


This analysis shows that in the US, the highest revenues come from Rock, Latin, and Metal genres. To maximize profits, marketing and sales activities should focus on these music genres.

## Top artists vs tracks

In [5]:
%%sql SELECT a.Name, SUM(Quantity) AS QuantityTracs
    FROM artists a
JOIN albums al
ON a.ArtistId = al.ArtistId
JOIN tracks t
     ON al.AlbumId = t.AlbumId
JOIN invoice_items ii on t.TrackId = ii.TrackId
GROUP BY a.Name
ORDER BY QuantityTracs DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

Name,QuantityTracs
Iron Maiden,140
U2,107
Metallica,91
Led Zeppelin,87
Os Paralamas Do Sucesso,45
Deep Purple,44
Faith No More,42
Lost,41
Eric Clapton,40
R.E.M.,39


Sales of songs are mainly concentrated on a few artists; in order to diversify revenue sources, it is necessary to ensure exposure for artists in the lower segment.

## Sales effectiveness of employees

In [6]:
%%sql SELECT
    e.FirstName,
    e.LastName,
    e.HireDate,
    COUNT(DISTINCT c.CustomerId) AS NumberOfCustomersSupported,
    SUM(i.Total) AS TotalSalesSupported
FROM
    employees AS e
JOIN
    customers AS c ON e.EmployeeId = c.SupportRepId
JOIN
    invoices AS i ON c.CustomerId = i.CustomerId
WHERE
    e.Title = 'Sales Support Agent'
GROUP BY
    e.EmployeeId, e.FirstName, e.LastName, e.HireDate
ORDER BY
    TotalSalesSupported DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

FirstName,LastName,HireDate,NumberOfCustomersSupported,TotalSalesSupported
Jane,Peacock,2002-04-01 00:00:00,21,833.04
Margaret,Park,2003-05-03 00:00:00,20,775.4
Steve,Johnson,2003-10-17 00:00:00,18,720.16


Support department employees do not differ significantly in terms of sales effectiveness. The analysis suggests that longer tenure leads to better results.

## Customers vs Revenue

In [7]:
%%sql SELECT FirstName, LastName, SUM(Total) AS SumSales, ROUND(AVG(Total),2)AS AverageSales, max(Total) AS MaxSales
    FROM customers
JOIN invoices
ON customers.CustomerId = invoices.CustomerId
JOIN invoice_items
ON invoices.InvoiceId = invoice_items.InvoiceId
GROUP BY FirstName, LastName
ORDER BY SumSales DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

FirstName,LastName,SumSales,AverageSales,MaxSales
Helena,Holý,502.62,13.23,25.86
Richard,Cunningham,474.62,12.49,23.86
Hugh,O'Reilly,446.62,11.75,21.86
Ladislav,Kovács,446.62,11.75,21.86
Luis,Rojas,415.62,10.94,17.91
Astrid,Gruber,404.62,10.65,18.86
Victor,Stevens,404.62,10.65,18.86
Fynn,Zimmermann,388.62,10.23,14.91
Frank,Ralston,378.62,9.96,15.86
František,Wichterlová,376.62,9.91,16.86


Customer vs. Revenue analysis shows which customers generate the most revenue and which generate the least, helping you build a strategy to increase customer engagement.

## Average Track Per Album For Artist

In [8]:
%%sql SELECT ar.Name, ROUND(AVG(AlbumTrackCount),2) AS AVGTrackPerAlbum
       FROM
           (SELECT
           a.AlbumId, COUNT(TrackId) AS AlbumTrackCount, a.ArtistId
       FROM tracks
           JOIN albums a
           ON tracks.AlbumId = a.AlbumId
       GROUP BY a.AlbumId) AlbumStats
JOIN artists ar
ON ar.ArtistId= AlbumStats.ArtistId
GROUP BY ar.Name
ORDER BY AVGTrackPerAlbum DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

Name,AVGTrackPerAlbum
Lenny Kravitz,57.0
Chico Buarque,34.0
Frank Sinatra,24.0
Eric Clapton,24.0
Battlestar Galactica (Classic),24.0
Lost,23.0
Heroes,23.0
Gene Krupa,22.0
The Who,20.0
James Brown,20.0


Lenny Kravitz is an artist who create the most songs per album

## Average length and average price in each genre

In [9]:
%%sql SELECT
    g.Name AS Genre,
    ROUND(AVG(t.Milliseconds),2) AS AvgLength,
    ROUND(AVG(t.UnitPrice),2) AS AvgPrice
FROM tracks t
JOIN genres g ON g.GenreId = t.GenreId
GROUP BY g.GenreId, g.Name
ORDER BY AvgPrice DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

Genre,AvgLength,AvgPrice
Science Fiction,2625549.08,1.99
TV Shows,2145041.02,1.99
Sci Fi & Fantasy,2911783.04,1.99
Drama,2575283.78,1.99
Comedy,1585263.71,1.99
Rock,283910.04,0.99
Jazz,291755.38,0.99
Metal,309749.44,0.99
Alternative & Punk,234353.85,0.99
Rock And Roll,134643.5,0.99


This analysis shows that the longer the average, the higher the price.

##  Which employee has the most customers from the Top 10 in terms of spending?

In [10]:
%%sql WITH Top10Customers AS (
    SELECT
        c.CustomerId,
        SUM(i.Total) AS TotalSpent
    FROM customers c
    JOIN invoices i ON c.CustomerId = i.CustomerId
    GROUP BY c.CustomerId
    ORDER BY TotalSpent DESC
    LIMIT 10
)
SELECT
    e.EmployeeId,
    e.FirstName,
    e.LastName,
    COUNT(t.CustomerId) AS NumberOfTopClients
FROM employees e
JOIN customers c ON c.SupportRepId = e.EmployeeId
JOIN Top10Customers t ON t.CustomerId = c.CustomerId
GROUP BY e.EmployeeId, e.FirstName, e.LastName
ORDER BY NumberOfTopClients DESC
LIMIT 1

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

EmployeeId,FirstName,LastName,NumberOfTopClients
5,Steve,Johnson,5


Our best employee is Steve Johnson

## Which file formats sell best

In [11]:
%%sql SELECT m.Name,
       ROUND(SUM(i.UnitPrice*i.Quantity),2) AS TotalRevenue,
       SUM(i.Quantity) AS TotalUnitsSold
    FROM media_types m
JOIN tracks t
ON m.MediaTypeId = t.MediaTypeId
JOIN invoice_items i
ON t.TrackId = i.TrackId
GROUP BY m.Name
ORDER BY TotalRevenue DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

Name,TotalRevenue,TotalUnitsSold
MPEG audio file,1956.24,1976
Protected MPEG-4 video file,220.89,111
Protected AAC audio file,144.54,146
Purchased AAC audio file,3.96,4
AAC audio file,2.97,3


Sales are clearly dominated by one file format, which generates significantly higher purchase volumes than all other types combined. Investments in promotion and catalog development should focus primarily on this best-selling format.

OSTATNIE
To chyba coś źle liczy
--Które albumy mają największy udział w sprzedaży swoich artystów?”

In [12]:
%%sql
WITH AlbumSales AS(
    SELECT
        a.AlbumId,
        a.Title AS AlbumTitle,
        ar.ArtistId,
        ar.Name AS ArtistName,
        SUM(il.UnitPrice * il.Quantity) AS AlbumRevenue
    FROM albums a
    JOIN artists ar ON ar.ArtistId = a.ArtistId
    JOIN tracks t ON t.AlbumId = a.AlbumId
    JOIN invoice_items il ON il.TrackId = t.TrackId
    GROUP BY a.AlbumId, a.Title, ar.ArtistId, ar.Name
),

ArtistSales AS (
    SELECT
        ArtistId,
        SUM(AlbumRevenue) AS ArtistRevenue
    FROM AlbumSales
    GROUP BY ArtistId
)

SELECT
    ASales.ArtistName,
    ASales.AlbumTitle,
    ASales.AlbumRevenue,
    ARSales.ArtistRevenue,
    ROUND(ASales.AlbumRevenue * 100.0 / ARSales.ArtistRevenue, 2)
        AS RevenueSharePercent
FROM AlbumSales AS ASales
JOIN ArtistSales AS ARSales
    ON ASales.ArtistId = ARSales.ArtistId
ORDER BY RevenueSharePercent DESC

Running query in 'sqlite:///C:/Users/Eva/OneDrive/Downloads/chinook/chinook.db'

ArtistName,AlbumTitle,AlbumRevenue,ArtistRevenue,RevenueSharePercent
Aerosmith,Big Ones,9.9,9.9,100.0
Alanis Morissette,Jagged Little Pill,7.92,7.92,100.0
Alice In Chains,Facelift,6.93,6.93,100.0
Apocalyptica,Plays Metallica By Four Cellos,3.96,3.96,100.0
BackBeat,BackBeat Soundtrack,5.9399999999999995,5.9399999999999995,100.0
Billy Cobham,The Best Of Billy Cobham,3.96,3.96,100.0
Body Count,Body Count,10.89,10.89,100.0
Bruce Dickinson,Chemical Wedding,11.879999999999999,11.879999999999999,100.0
Buddy Guy,The Best Of Buddy Guy - The Millenium Collection,6.93,6.93,100.0
Chico Buarque,Minha Historia,26.73,26.73,100.0
